# Importing required  packages

In [0]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, RidgeClassifier
from time import time
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,mean_squared_log_error,r2_score
from sklearn.model_selection import KFold ,cross_validate, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, ParameterGrid
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC, SVR
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error
from time import time
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
!pip install catboost
from catboost import  CatBoostRegressor


# from keras.layers import Dense,Dropout
# from keras import Sequential

     |████████████████████████████████| 61.2MB 511kB/s 


# Data Preparation

In [0]:
data_df=pd.read_csv("https://raw.githubusercontent.com/Kadri314/cadmium_data_science/master/data.csv")
print(data_df.columns)
print(data_df.shape)
print(data_df.info()) # we can notice from this that: type5industry 30 non-null float64 has one null value
data_df.head(2)

Index(['Spring', 'aquifer', 'Cd_con', 'logged_cd', 'Type1industry',
       'Type2industry', 'type3industry', 'type4industry', 'type5industry',
       'type6industry', 'type7industry', 'type8industry', 'type9industry',
       'Total_industry', 'MSWdumps', 'Multipledumps', 'Total_dump',
       'burningdump ', 'NoBurningdump', 'Operational', 'Non_operational',
       'Type1agri', 'Type2agri', 'type3agri', 'type4agri', 'type5agri',
       'type6agri', 'type7agri', 'type8agri', 'type9agri', 'type10agri',
       'type11agri', 'type12agri', 'type13agri', 'type14agri', 'type15agri',
       'Total_agri', 'Mean_disindustry', 'Median_disindustry',
       'Min_disindustry', 'Max_disindustry', 'Mean_yrs', 'Median_yrs',
       'Min_yrs', 'Max_yrs', 'Mean_volume', 'Median_volume', 'Min_volume',
       'Max_volume', 'Mean_area', 'Median_area', 'Min_area', 'Max_area',
       'Mean_distdump', 'Median_distdump', 'Min_distdump', 'Max_distdump',
       'Mean_areaagri', 'Median_areaagri', 'Min_areaagri', 'M

,Spring,aquifer,Cd_con,logged_cd,Type1industry,Type2industry,type3industry,type4industry,type5industry,type6industry,type7industry,type8industry,type9industry,Total_industry,MSWdumps,Multipledumps,Total_dump,burningdump,NoBurningdump,Operational,Non_operational,Type1agri,Type2agri,type3agri,type4agri,type5agri,type6agri,type7agri,type8agri,type9agri,type10agri,type11agri,type12agri,type13agri,type14agri,type15agri,Total_agri,Mean_disindustry,Median_disindustry,Min_disindustry,Max_disindustry,Mean_yrs,Median_yrs,Min_yrs,Max_yrs,Mean_volume,Median_volume,Min_volume,Max_volume,Mean_area,Median_area,Min_area,Max_area,Mean_distdump,Median_distdump,Min_distdump,Max_distdump,Mean_areaagri,Median_areaagri,Min_areaagri,Max_areaagri,Mean_distagri,Median_distagri,Min_distagri,Max_distagri,Distance_Zouk,Slope,Precipitation,Traffic,SlopeInd
0,ras el haref,27b,1.0,0.0,1,0,0,0,1.0,4,3,1,0,10,10,2,12,1,11,10,2,27,7,0,8,746,342,65,397,5,0,37,81,19,2,45,1781,7468.10,6974.81,1717.59,10925.57,6.67,3.0,3,23,996.88,200,10.0,5100,1029.17,300,20,5100,8394.92,7768.15,3330.45,17487.87,22754.45,5146.0,236,1965571,10768.59,9246.53,322.76,24805.02,36.7,-0.92,1200,0,-0.90
1,Ain Jrain,28,1.0,0.0,0,1,0,0,0.0,0,0,0,0,1,1,1,2,0,2,1,1,35,32,0,25,1317,963,122,952,51,5,106,221,47,13,91,3980,16872.43,16872.43,16872.43,16872.43,10.50,10.5,3,18,1250.00,1250,1000.0,1500,875.00,875,750,1000,18669.39,18669.39,5962.25,31376.52,24059.80,4285.5,91,3957842,33258.78,36155.12,203.77,58814.80,30.7,-0.69,1500,0,-0.67


In [0]:
#checking in which sample is type5industry has the value null
data_df[pd.isnull(data_df).any(axis=1)] # yes indeed type5industry is null with sample number 13

,Spring,aquifer,Cd_con,logged_cd,Type1industry,Type2industry,type3industry,type4industry,type5industry,type6industry,type7industry,type8industry,type9industry,Total_industry,MSWdumps,Multipledumps,Total_dump,burningdump,NoBurningdump,Operational,Non_operational,Type1agri,Type2agri,type3agri,type4agri,type5agri,type6agri,type7agri,type8agri,type9agri,type10agri,type11agri,type12agri,type13agri,type14agri,type15agri,Total_agri,Mean_disindustry,Median_disindustry,Min_disindustry,Max_disindustry,Mean_yrs,Median_yrs,Min_yrs,Max_yrs,Mean_volume,Median_volume,Min_volume,Max_volume,Mean_area,Median_area,Min_area,Max_area,Mean_distdump,Median_distdump,Min_distdump,Max_distdump,Mean_areaagri,Median_areaagri,Min_areaagri,Max_areaagri,Mean_distagri,Median_distagri,Min_distagri,Max_distagri,Distance_Zouk,Slope,Precipitation,Traffic,SlopeInd
13,Ain el saydeh,28,9.9445,2.3,2,5,3,2,NaN,5,9,7,2,35,14,3,17,2,15,14,3,35,32,0,25,1317,963,122,952,51,5,106,221,47,13,91,3980,8257.03,3506.59,2314.03,36288.37,5.18,3.0,3,18,2151.76,1000,15.0,20000,1303.82,500,15,10000,12146.25,11446.22,2343.56,32374.55,24059.8,4285.5,91,3957842,16902.55,12080.35,112.02,48746.46,28.5,-1.15,1200,1,-1.12


In [0]:
# Replacing the value of type5industry with Nan value with 0 value as if the spring has 0 type5industry
data_df['type5industry'].fillna(0, inplace=True)
data_df[pd.isnull(data_df).any(axis=1)] # check if null value exist after replacing type5industry

,Spring,aquifer,Cd_con,logged_cd,Type1industry,Type2industry,type3industry,type4industry,type5industry,type6industry,type7industry,type8industry,type9industry,Total_industry,MSWdumps,Multipledumps,Total_dump,burningdump,NoBurningdump,Operational,Non_operational,Type1agri,Type2agri,type3agri,type4agri,type5agri,type6agri,type7agri,type8agri,type9agri,type10agri,type11agri,type12agri,type13agri,type14agri,type15agri,Total_agri,Mean_disindustry,Median_disindustry,Min_disindustry,Max_disindustry,Mean_yrs,Median_yrs,Min_yrs,Max_yrs,Mean_volume,Median_volume,Min_volume,Max_volume,Mean_area,Median_area,Min_area,Max_area,Mean_distdump,Median_distdump,Min_distdump,Max_distdump,Mean_areaagri,Median_areaagri,Min_areaagri,Max_areaagri,Mean_distagri,Median_distagri,Min_distagri,Max_distagri,Distance_Zouk,Slope,Precipitation,Traffic,SlopeInd


In [0]:
#Split the data into training and testin
X=data_df[data_df.columns.difference(['Spring', 'aquifer', 'Cd_con', 'logged_cd'])]# to omit the columns:  'Spring', 'aquifer', 'Cd_con', 'logged_cd'
y=data_df["Cd_con"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(24, 66) (24,) (7, 66) (7,)


In [0]:
# standardize data
scaler = preprocessing.StandardScaler().fit(X_train) # calculate the mean and std only on the training data 
print(scaler.mean_) #mean of each feature in   the training data
print(scaler.var_ ) # variance of each feature in the training data
X_train= scaler.transform(X_train) #standardize training data 
X_test= scaler.transform(X_test) # standardize testing data 


[ 3.70500000e+01  7.12500000e+00  4.10625000e+03  3.34359304e+06
  2.58509692e+04  4.70753367e+04  2.51374667e+04  7.27291667e+03
  1.65833333e+01  8.61711250e+02  2.29599937e+04  1.18658921e+04
  1.97628717e+04  1.12936408e+04  1.26411167e+03  6.71583333e+00
  5.03125000e+02  4.46772917e+03  9.80962042e+03  1.74521838e+04
  9.98919458e+03  8.40000000e+02  4.52083333e+00  2.06500000e+02
  1.19333333e+02  5.59781417e+03  2.03738333e+02  3.04114667e+03
  3.19658333e+02  3.79166667e+00  2.16666667e+00  8.16666667e+00
  2.08333333e+00  7.12500000e+00  1.25000000e+03 -9.32083333e-01
 -9.41250000e-01  3.23962500e+03  9.29166667e+00  1.47500000e+01
  8.33333333e-01  3.27083333e+01  8.33333333e-01  2.64166667e+01
  2.20833333e+00  1.04166667e+00  3.95833333e+00  9.15833333e+01
  1.79791667e+02  3.62500000e+01  1.15000000e+01  7.37500000e+01
  8.75000000e-01  1.04166667e+00  1.97500000e+01  7.50000000e-01
  1.08400000e+03  2.91666667e-01  7.74000000e+02  2.54166667e+00
  1.23875000e+02  4.00000

# Feature selection using PCA with one leave out corss validation 

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
x_new = pca.fit_transform(X_train)
print(pca.explained_variance_ratio_)
print(abs( pca.components_ )[0])


[0.3666501  0.20852002 0.14194531 0.07510119 0.04051139]
[0.110126   0.14691323 0.15539648 0.12313377 0.18426324 0.11707882
 0.17540863 0.16084202 0.07638356 0.15403769 0.08908131 0.11194244
 0.00058791 0.13985707 0.1578536  0.01109195 0.0744213  0.1214613
 0.06803092 0.05979708 0.10710727 0.10041889 0.00668684 0.02703321
 0.11270335 0.01875749 0.05876492 0.0495241  0.03281463 0.0033526
 0.17234572 0.15588363 0.09778839 0.15308102 0.04044659 0.00900464
 0.00825382 0.160638   0.15524958 0.17345966 0.02736287 0.09672623
 0.15212309 0.13518461 0.18157354 0.14707549 0.11577815 0.10927337
 0.15654952 0.15635025 0.0804267  0.16183717 0.03838021 0.16557888
 0.15287972 0.1657405  0.16196476 0.04870334 0.15991644 0.1325765
 0.01259944 0.17160752 0.15708205 0.15869411 0.15048699 0.14737079]


In [0]:
test=[100,90,200,10,1000,1]
indices = np.argsort(test)[::-1]
print(indices)

[4 2 0 1 3 5]


we can see that PC1 explains 36%, PC2 20%, PC3 14%, PC4 7%, PC5=4%, PC6=3.6 % and so on .... 
<br/>I will be using only the first Principle component for feature selection process.

In [0]:
#I will pick only the top 10 features
loo = LeaveOneOut()
loo.get_n_splits(X_train)
PCA_scores={} # key is the feature index, value is the score (total votes)
for train_index, test_index in loo.split(X_train):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
    y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
    pca = PCA(n_components=1)
    x_new = pca.fit_transform(X_train_cross)
    indices=np.argsort(abs( pca.components_ )[0])[::-1][:4] # get the indices of the top 4 voted features in sorted order
     # accumualte the vote scores for all cross validation 
    for index_feaure in indices:
        if index_feaure in PCA_scores:
            PCA_scores[index_feaure]= PCA_scores[index_feaure]+1
        else:
            PCA_scores[index_feaure]=1

PCA_scores=dict(sorted(PCA_scores.items(), key=lambda x: x[1], reverse=True)) #sort by top voted
print(PCA_scores)

{44: 22, 4: 22, 6: 16, 39: 13, 30: 7, 56: 6, 51: 4, 61: 2, 53: 2, 14: 1, 37: 1}


We can see that feature with index 44 has been voted to be top feature 22 times <br/>
Feature with index 4 has been voted to be top feature 22 times and so on ....

In [0]:
top_features_PCA=list(PCA_scores.keys())
X.columns[top_features_PCA] #list top voted features by PCA

Index(['Type2industry', 'Max_disindustry', 'Max_distdump', 'Total_industry',
       'Multipledumps', 'type5agri', 'type15agri', 'type7industry',
       'type3industry', 'Mean_volume', 'Total_agri'],
      dtype='object')

# Feature selection using Decision trees with one leave out corss validation 

In [0]:
from sklearn.ensemble import ExtraTreesRegressor

DT_scores={} # key is the feature index, value is the score (total votes)
loo = LeaveOneOut()
loo.get_n_splits(X_train)
for train_index, test_index in loo.split(X_train):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
    y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
    
    forest = ExtraTreesRegressor(n_estimators=100,
                              random_state=0)

    forest.fit(X_train_cross, y_train_cross)
    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1][:4] # get the indices of the top 4 voted features in sorted order
    # accumualte the vote scores for all cross validation 
    for index_feaure in indices:
        if index_feaure in DT_scores:
            DT_scores[index_feaure]= DT_scores[index_feaure]+1
        else:
            DT_scores[index_feaure]=1
            
DT_scores=dict(sorted(DT_scores.items(), key=lambda x: x[1], reverse=True)) #sort by top voted
print(DT_scores)


{40: 24, 35: 17, 34: 15, 26: 10, 36: 10, 14: 4, 6: 3, 2: 2, 8: 2, 19: 2, 11: 1, 29: 1, 0: 1, 18: 1, 57: 1, 59: 1, 27: 1}


We can see that feature with index 44 has been voted to be top feature 24 times <br/>
Feature with index 35 has been voted to be top feature 17 times and so on ....

In [0]:
print("Top features by decision trees: ")
top_features_DT=list(DT_scores.keys())
X.columns[top_features_DT] #list top voted features by decision trees

Top features by decision trees: 


Index(['Traffic', 'Slope', 'Precipitation', 'Min_distagri', 'SlopeInd',
       'Mean_volume', 'Max_distdump', 'Max_area', 'Max_yrs', 'Median_distagri',
       'Mean_disindustry', 'Min_yrs', 'Distance_Zouk', 'Median_disindustry',
       'type5industry', 'type6industry', 'Min_distdump'],
      dtype='object')

# Feature selection using univariate selection with one leave out corss validation


In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression


from sklearn.ensemble import ExtraTreesRegressor

UV_scores={} # key is the feature index, value is the score (total votes)
loo = LeaveOneOut()
loo.get_n_splits(X_train)
for train_index, test_index in loo.split(X_train):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
    y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
    
    UV = SelectKBest(score_func=f_regression, k=10)

    UV.fit(X_train_cross, y_train_cross)
    importances = UV.scores_
    indices = np.argsort(importances)[::-1][:4] # get the indices of the top 4 voted features in sorted order
    # accumualte the vote scores for all cross validation 
    for index_feaure in indices:
        if index_feaure in UV_scores:
            UV_scores[index_feaure]= UV_scores[index_feaure]+1
        else:
            UV_scores[index_feaure]=1
            
UV_scores=dict(sorted(UV_scores.items(), key=lambda x: x[1], reverse=True)) #sort by top voted
print(UV_scores)


{40: 24, 13: 23, 20: 22, 6: 12, 34: 7, 19: 3, 8: 2, 52: 1, 9: 1, 57: 1}


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:293: RuntimeWarning: invalid value encountered in sqrt
  n_samples * X_means ** 2)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [0]:
print("Top features by univariate selection: ")
top_features_UV=list(UV_scores.keys())
X.columns[top_features_UV] #list top voted features by decision trees

Top features by univariate selection: 


Index(['Traffic', 'Mean_distdump', 'Median_distdump', 'Max_distdump',
       'Precipitation', 'Median_distagri', 'Max_yrs', 'type3agri', 'Mean_area',
       'type5industry'],
      dtype='object')

# Feature selection using Recursive Feature Elimination with one leave out corss validation


In [0]:
# Feature Extraction with RFE
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression


RFE_scores={} # key is the feature index, value is the score (total votes)
loo = LeaveOneOut()
loo.get_n_splits(X_train)
for train_index, test_index in loo.split(X_train):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
    y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
    
        # feature extraction
    model = LinearRegression()
    rfe = RFE(model)
    fit = rfe.fit(X_train_cross, y_train_cross)

    # accumualte the vote scores for all cross validation 
    for feature_index in range(0,len(fit.ranking_)):
        if feature_index in RFE_scores:
            RFE_scores[feature_index]=RFE_scores[feature_index]+fit.ranking_[feature_index]
        else:
            RFE_scores[feature_index]=fit.ranking_[feature_index]
            
RFE_scores=dict(sorted(RFE_scores.items(), key=lambda x: x[1], reverse=False)) #sort by top voted
print(RFE_scores)




{13: 24, 40: 24, 51: 24, 56: 24, 26: 28, 55: 28, 24: 34, 53: 39, 63: 43, 65: 43, 30: 48, 37: 48, 11: 51, 61: 59, 41: 60, 0: 67, 45: 71, 49: 80, 14: 82, 25: 86, 17: 87, 3: 88, 1: 92, 4: 99, 42: 100, 5: 105, 38: 114, 58: 116, 31: 120, 59: 122, 33: 130, 27: 136, 52: 138, 22: 143, 34: 150, 7: 157, 23: 165, 28: 172, 48: 230, 8: 231, 62: 237, 18: 254, 46: 265, 12: 281, 20: 294, 19: 298, 10: 308, 6: 312, 15: 341, 39: 361, 2: 417, 16: 447, 9: 451, 32: 451, 29: 464, 60: 478, 36: 491, 35: 501, 57: 523, 43: 532, 21: 562, 64: 565, 44: 598, 50: 643, 54: 645, 47: 671}


In [0]:
# top 10 selected feature by RFE
print("Top features by RFE: ")
top_features_RFE=list(RFE_scores.keys())
X.columns[top_features_RFE][:10] #list top voted features by decision trees

Top features by RFE: 


Index(['Mean_distdump', 'Traffic', 'type15agri', 'type5agri', 'Min_distagri',
       'type4industry', 'Min_areaagri', 'type3industry', 'type8industry',
       'type9industry'],
      dtype='object')

In [0]:
#taking the intersection between all features_model
set_a= list(set(top_features_RFE).intersection(set(top_features_UV)))
set_b= list(set(set_a).intersection(set(top_features_DT)))
set_c= list(set(set_b).intersection(set(top_features_PCA)))
X.columns[set_c]


Index(['Max_distdump'], dtype='object')

top selected feature(s) for the intersection between all models is only  'Max_distdump'. <br/>
I will take the set of top 3 features from each model: 

In [0]:
top_features=set(top_features_UV[:2]+top_features_RFE[:2]+top_features_DT[:2]+top_features_PCA[:2])
top_features=list(top_features)
print("Top features:")
print(X.columns[top_features])



Top features:
Index(['Slope', 'Max_disindustry', 'Traffic', 'Type2industry',
       'Mean_distdump'],
      dtype='object')


In [0]:
X_train=X_train[:,top_features]
X_test=X_test[:,top_features]
print(X_train.shape,X_test.shape)

(24, 5) (7, 5)


In [0]:
#Define MAPE error function: 
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Tuning hyper-parameters on the top selected features + modeling


# SVM with polinomial kernal

In [0]:
current=time()

 # gird of parameters for poly kernal
tuned_parameters = [{'kernel': ['poly'], 'gamma': [1e-3,1e-6,1e-2,1e-1,1,10,100],'degree':[2,3,4,5,6,7],
                     'C': [0.0001,0.01,0.1,0.5,1,3,5,10,1000]}]


best_parameters=None
top_mean_rmse_error=10000000
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    svm= SVR(**g)
    mse_scores=[]
    mae_scores=[]
    rmse_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        svm.fit(X_train_cross,y_train_cross)
        pred=svm.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)
        
    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores)
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for SVM with polynomial kernal:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))

# Tuning hyper-parameters

It took 104.26081832647324 mins
Best parameters for SVM with polynomial kernal:
{'C': 0.1, 'degree': 3, 'gamma': 1, 'kernel': 'poly'}
With mean score of MSE: 327.96722913323015
With mean score of MAE: 11.468113719997291
With mean score of RMSE: 11.468113719997291
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.8465734704461704


# SVM with linear kernal


In [0]:
current=time()

#create the model 
poly_svr= SVR()

#gird of parameters for RBF kernal
tuned_parameters = [{'kernel': ['linear'],'C': [0.0001,0.001,0.01,0.1,0.5,1,3,5,10,1000]}]

best_parameters=None
top_mean_rmse_error=10000000
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    svm= SVR(**g)
    mse_scores=[]
    mae_scores=[]
    rmse_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        svm.fit(X_train_cross,y_train_cross)
        pred=svm.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores)
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for SVM with linear kernal:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_mae_error)
# print("With mean score of MAPE:",top_mean_mae_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))

# Tuning hyper-parameters

It took 0.015471903483072917 mins
Best parameters for SVM with linear kernal:
{'C': 1000, 'kernel': 'linear'}
With mean score of MSE: 488.3853739489868
With mean score of MAE: 12.588535952621989
With mean score of RMSE: 12.588535952621989
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.7377745374039979


# SVM with RBF kernal

In [0]:
current=time()

# gird of parameters for RBF kernal 
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3,1e-5,1e-6,1e-7, 1e-4,1e-2,1e-1,1,10,100],
                     'C': [0.0001,0.001,0.01,0.1,0.5,1,3,5,10,1000]}]


best_parameters=None
top_mean_rmse_error=10000000
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    svm= SVR(**g)
    mse_scores=[]
    mae_scores=[]
    rmse_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        svm.fit(X_train_cross,y_train_cross)
        pred=svm.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores)
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for SVM with RBF kernal:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_mae_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))

# Tuning hyper-parameters

It took 0.08639915386835734 mins
Best parameters for SVM with RBF kernal:
{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
With mean score of MSE: 298.02834673182457
With mean score of MAE: 11.668780013203614
With mean score of RMSE: 11.668780013203614
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.8269223752818368


# Random Forest


In [0]:

current=time()

# gird of parameters for random forest trees 
tuned_parameters = [{'n_estimators': [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,
                                      210,220,230,240,250,260,270,280,290,300],
                     'random_state': [30],
                     'criterion': ["mae","mse"],
                     'bootstrap': [True,False]}]


best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    model= RandomForestRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for random forest:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))

# Tuning hyper-parameters

It took 5.1234893798828125 mins
Best parameters for M5 regression trees:
{'bootstrap': True, 'criterion': 'mse', 'n_estimators': 200, 'random_state': 30}
With mean score of MSE: 353.421940114258
With mean score of MAE: 9.547843500000003
With mean score of RMSE: 9.547843500000003
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.8048902193180997


# M5 regression trees

In [0]:
#Source : https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html


current=time()

# gird of parameters for M5 regression trees 
tuned_parameters = [{'max_features': ["auto","sqrt","log2",None],
                     'random_state': [30],
                     'presort': [True,False],
                     'criterion': ["mae","mse"]}]


best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    model= DecisionTreeRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for M5 regression trees:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))

# Tuning hyper-parameters

It took 0.015040377775828043 mins
Best parameters for M5 regression trees:
{'criterion': 'mae', 'max_features': 'auto', 'presort': True, 'random_state': 30}
With mean score of MSE: 352.0632925104167
With mean score of MAE: 10.450620833333334
With mean score of RMSE: 10.450620833333334
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.7842051405185578


# GradientBoostingRegressor (GBDT )

In [0]:
#Source : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

current=time()

# gird of parameters for GBDT
tuned_parameters = [{'loss': ['ls', 'lad' ,'huber'],   
                     'learning_rate' : [0.001,0.01,0.1],
                     'n_estimators' : [10,40,60,100,160],
                     'random_state': [30],
                     'criterion': ["friedman_mse"],
                     'max_features': ["auto","sqrt","log2"],
                     'max_depth': [3,6,9,12,16]
                    }]




best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    model= GradientBoostingRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for GBDT:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))

# Tuning hyper-parameters

It took 18.799745798110962 mins
Best parameters for GBDT:
{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'huber', 'max_depth': 3, 'max_features': 'auto', 'n_estimators': 160, 'random_state': 30}
With mean score of MSE: 290.84976958856276
With mean score of MAE: 9.133561697742168
With mean score of RMSE: 9.133561697742168
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.8291035561011352


# CatBoostRegressor 

In [0]:
#Source : https://catboost.ai/docs/concepts/python-reference_catboostregressor.html


current=time()

# gird of parameters for CatBoostRegressor
tuned_parameters = [{'depth':[3,6,12], 
          #'iterations':[250,100,500],
          'learning_rate':[0.001,0.01,0.1,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
#           'border_count':[32,5,10,20,50,100,200],
          #'thread_count':[4],
           'verbose':[0]}]


best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

counter =0
print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    counter=counter+1
    if counter %10 ==0:
      print(counter)
    # for each possible combination in the grid
    model= CatBoostRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds

print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for CatBoostRegressor:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))


# Tuning hyper-parameters

10
20
30
40
50
60
It took 30.592853625615437 mins
Best parameters for CatBoostRegressor:
{'depth': 3, 'l2_leaf_reg': 10, 'learning_rate': 0.01, 'verbose': 0}
With mean score of MSE: 654.9698620666114
With mean score of MAE: 11.403836548977162
With mean score of RMSE: 11.403836548977162
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.5747472009449256


# Picking the final Model for top feaures

In [0]:
print("Cad concentration mean:",np.mean(y))
print("---------------------------------------------------\n")  

# retrain all the models on the best tuned hyper-parameters
svm_poly=SVR(**{'C': 0.1, 'degree': 3, 'gamma': 1, 'kernel': 'poly'})
svm_poly.fit(X_train,y_train)
preds=svm_poly.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("SVM with polinomial kernel, results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")  

svm_rbf=SVR(**{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'})
svm_rbf.fit(X_train,y_train)
preds=svm_rbf.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("SVM with RBF kernel, results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n") 

svm_linear=SVR(**{'C': 1000, 'kernel': 'linear'})
svm_linear.fit(X_train,y_train)
preds=svm_linear.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("SVM with linear kernel, results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")

randomForest=RandomForestRegressor(**{'bootstrap': True, 'criterion': 'mse', 'n_estimators': 200, 'random_state': 30})
randomForest.fit(X_train,y_train)
preds=randomForest.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("Random forest results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")


m5_regression=DecisionTreeRegressor(**{'criterion': 'mae', 'max_features': 'auto', 'presort': True, 'random_state': 30})
m5_regression.fit(X_train,y_train)
preds=m5_regression.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("M5 regression trees results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")

# to add optimal paraeters: 

GBDT=GradientBoostingRegressor(**{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'huber', 'max_depth': 3, 'max_features': 'auto', 'n_estimators': 160, 'random_state': 30})
GBDT.fit(X_train,y_train)
preds=GBDT.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("GBDT results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")

catBostReg=CatBoostRegressor(**{'depth': 3, 'l2_leaf_reg': 10, 'learning_rate': 0.01, 'verbose': 0})
catBostReg.fit(X_train,y_train)
preds=catBostReg.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("CatBoostRegressor results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")



Cad concentration mean: 24.001398387096778
---------------------------------------------------

SVM with polinomial kernel, results on the testing data set:
MSE: 1601.896559309708
RMSE:  40.02369997026397
MAE: 30.542780930836972
MAPE:  150.7009683122459
R2:  -0.40230539014173283
correlation between y_true and y_preds: 0.4039695868031746
---------------------------------------------------

SVM with RBF kernel, results on the testing data set:
MSE: 574.9250235700468
RMSE:  23.977594198960972
MAE: 17.414043228753613
MAPE:  60.894486367151416
R2:  0.4967087888452314
correlation between y_true and y_preds: 0.7157050595787596
---------------------------------------------------

SVM with linear kernel, results on the testing data set:
MSE: 787.646002646928
RMSE:  28.065031670157225
MAE: 18.888877365171634
MAPE:  59.44890752305951
R2:  0.310492160922464
correlation between y_true and y_preds: 0.6854404724309975
---------------------------------------------------

Random forest results on the t

# Tuning Hyper-Parameters on all features + modeling


In [0]:
print(X_train.shape,X_test.shape)

(24, 66) (7, 66)


# SVM with polinomial kernal (all Features)

In [0]:
current=time()

 # gird of parameters for poly kernal
tuned_parameters = [{'kernel': ['poly'], 'gamma': [1e-3,1e-6,1e-2,1e-1,1,10,100],'degree':[2,3,4,5,6,7],
                     'C': [0.0001,0.01,0.1,0.5,1,3,5,10,1000]}]


best_parameters=None
top_mean_rmse_error=10000000
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    svm= SVR(**g)
    mse_scores=[]
    mae_scores=[]
    rmse_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        svm.fit(X_train_cross,y_train_cross)
        pred=svm.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)
        
    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores)
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for SVM with polynomial kernal:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))

# Tuning hyper-parameters

It took 0.3864992380142212 mins
Best parameters for SVM with polynomial kernal:
{'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'}
With mean score of MSE: 867.9132021101668
With mean score of MAE: 16.750771480909307
With mean score of RMSE: 16.750771480909307
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.39150083859353524


# SVM with Linear kernal (all Features)

In [0]:
current=time()

#create the model 
poly_svr= SVR()

#gird of parameters for RBF kernal
tuned_parameters = [{'kernel': ['linear'],'C': [0.0001,0.001,0.01,0.1,0.5,1,3,5,10,1000]}]

best_parameters=None
top_mean_rmse_error=10000000
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    svm= SVR(**g)
    mse_scores=[]
    mae_scores=[]
    rmse_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        svm.fit(X_train_cross,y_train_cross)
        pred=svm.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores)
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for SVM with linear kernal:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_mae_error)
# print("With mean score of MAPE:",top_mean_mae_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))


# Tuning hyper-parameters

It took 0.011318596204121907 mins
Best parameters for SVM with linear kernal:
{'C': 0.1, 'kernel': 'linear'}
With mean score of MSE: 962.5069507168761
With mean score of MAE: 16.70524799774698
With mean score of RMSE: 16.70524799774698
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.16922168964960382


# SVM with RBF kernal (all Features)

In [0]:
current=time()

# gird of parameters for RBF kernal 
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3,1e-5,1e-6,1e-7, 1e-4,1e-2,1e-1,1,10,100],
                     'C': [0.0001,0.001,0.01,0.1,0.5,1,3,5,10,1000]}]


best_parameters=None
top_mean_rmse_error=10000000
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    svm= SVR(**g)
    mse_scores=[]
    mae_scores=[]
    rmse_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        svm.fit(X_train_cross,y_train_cross)
        pred=svm.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores)
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for SVM with RBF kernal:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_mae_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))



# Tuning hyper-parameters

It took 0.10126898288726807 mins
Best parameters for SVM with RBF kernal:
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
With mean score of MSE: 927.3482708413885
With mean score of MAE: 16.537486641633542
With mean score of RMSE: 16.537486641633542
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.3125755693823623


# Random Forest (all Features)


In [0]:

current=time()

# gird of parameters for random forest trees 
tuned_parameters = [{'n_estimators': [10,50,100,150,200,250,300],
                     'random_state': [30],
                     'criterion': ["mae","mse"],
                     'bootstrap': [True,False]}]


best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    model= RandomForestRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for random forest:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))



# Tuning hyper-parameters

It took 2.217187710603078 mins
Best parameters for random forest:
{'bootstrap': False, 'criterion': 'mae', 'n_estimators': 150, 'random_state': 30}
With mean score of MSE: 442.01966654953065
With mean score of MAE: 11.206886000000006
With mean score of RMSE: 11.206886000000006
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.7348613241488156


# M5 regression trees (all Features)

In [0]:
#Source : https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html


current=time()

# gird of parameters for M5 regression trees 
tuned_parameters = [{'max_features': ["auto","sqrt","log2",None],
                     'random_state': [30],
                     'presort': [True,False],
                     'criterion': ["mae","mse"]}]


best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    model= DecisionTreeRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for M5 regression trees:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))



# Tuning hyper-parameters

It took 0.019685184955596922 mins
Best parameters for M5 regression trees:
{'criterion': 'mae', 'max_features': 'auto', 'presort': True, 'random_state': 30}
With mean score of MSE: 397.1410775012501
With mean score of MAE: 9.717062500000003
With mean score of RMSE: 9.717062500000003
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.8169670697344171


#  GradientBoostingRegressor (GBDT ) (all Features)

In [0]:
#Source : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

current=time()

# gird of parameters for GBDT
tuned_parameters = [{'loss': ['ls', 'lad' ,'huber'],   
                     'learning_rate' : [0.001,0.01,0.1],
                     'n_estimators' : [10,40,60,100,160],
                     'random_state': [30],
                     'criterion': ["friedman_mse"],
                     'max_features': ["auto","sqrt","log2"],
                     'max_depth': [3,6,9,12,16]
                    }]




best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    # for each possible combination in the grid
    model= GradientBoostingRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds
    
      
print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for GBDT:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))


# Tuning hyper-parameters

It took 20.02120784521103 mins
Best parameters for GBDT:
{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'ls', 'max_depth': 6, 'max_features': 'auto', 'n_estimators': 160, 'random_state': 30}
With mean score of MSE: 444.04811887225
With mean score of MAE: 11.165770318059485
With mean score of RMSE: 11.165770318059485
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.7341056219196949


# CatBoostRegressor (all Features)

In [0]:
#Source : https://catboost.ai/docs/concepts/python-reference_catboostregressor.html


current=time()

# gird of parameters for CatBoostRegressor
tuned_parameters = [{'depth':[3,6,12], 
          #'iterations':[250,100,500],
          'learning_rate':[0.001,0.01,0.1,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
#           'border_count':[32,5,10,20,50,100,200],
          #'thread_count':[4],
           'verbose':[0]}]


best_parameters=None
top_mean_mse_error=10000000
top_mean_mae_error=10000000
top_mean_rmse_error=10000000
top_mean_mape_error=10000000
top_y_true=[]
top_preds=[]

counter =0
print("# Tuning hyper-parameters" )
print()
# Simulating grid search 
for g in ParameterGrid(tuned_parameters):
    counter=counter+1
    if counter %10 ==0:
      print(counter)
    # for each possible combination in the grid
    model= CatBoostRegressor(**g)
    mse_scores=[]
    rmse_scores=[]
    mae_scores=[]
    mape_scores=[]
    y_true=[]
    preds=[]
    # doing 1 leave out cross validation
    loo = LeaveOneOut()
    loo.get_n_splits(X_train)
    for train_index, test_index in loo.split(X_train):
        X_train_cross, X_test_cross = X_train[train_index,:], X_train[test_index,:]
        y_train_cross, y_test_cross = y_train[train_index], y_train[test_index]
        
        model.fit(X_train_cross,y_train_cross)
        pred=model.predict(X_test_cross)
        
        preds.append(pred)
        y_true.append(y_test_cross.values[0])
        
        mse=mean_squared_error(y_test_cross,pred)
        mae=mean_absolute_error(y_test_cross,pred)
        mape=mean_absolute_percentage_error(y_test_cross,pred)
        
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(np.sqrt(mse))
        mape_scores.append(mape)

    if np.mean(rmse_scores) < top_mean_rmse_error:
        top_mean_mse_error=np.mean(mse_scores) 
        top_mean_mae_error=np.mean(mae_scores) 
        top_mean_rmse_error=np.mean(rmse_scores)
        top_mean_mape_error=np.mean(mape_scores)
        best_parameters=g
        top_y_true=y_true
        top_preds=preds

print("It took "+ str((time()-current)/60)+" mins")
print("Best parameters for CatBoostRegressor:")
print(best_parameters)
print("With mean score of MSE:",top_mean_mse_error)
print("With mean score of MAE:",top_mean_mae_error)
print("With mean score of RMSE:",top_mean_rmse_error)
# print("With mean score of MAPE:",top_mean_mape_error)
print("Cad concentration mean:",np.mean(y_train))
df=pd.DataFrame(
    {'lst1Title': top_y_true,
     'lst2Title': np.reshape(np.array(top_preds),(len(top_preds)))
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))



# Tuning hyper-parameters

10
20
30
40
50
60
It took 95.9950494090716 mins
Best parameters for CatBoostRegressor:
{'depth': 3, 'l2_leaf_reg': 3, 'learning_rate': 0.3, 'verbose': 0}
With mean score of MSE: 426.4333574339784
With mean score of MAE: 11.448270151683566
With mean score of RMSE: 11.448270151683566
Cad concentration mean: 21.378179166666666
correlation between y_true and y_preds: 0.7592706679256364


# Picking the final Model for all feaures

In [0]:


print("Cad concentration mean:",np.mean(y))
print("---------------------------------------------------\n")  

# retrain all the models on the best tuned hyper-parameters
svm_poly=SVR(**{'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'})
svm_poly.fit(X_train,y_train)
preds=svm_poly.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("SVM with polinomial kernel, results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")  

svm_rbf=SVR(**{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'})
svm_rbf.fit(X_train,y_train)
preds=svm_rbf.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("SVM with RBF kernel, results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n") 

svm_linear=SVR(**{'C': 0.1, 'kernel': 'linear'})
svm_linear.fit(X_train,y_train)
preds=svm_linear.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("SVM with linear kernel, results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")

randomForest=RandomForestRegressor(**{'bootstrap': False, 'criterion': 'mae', 'n_estimators': 150, 'random_state': 30})
randomForest.fit(X_train,y_train)
preds=randomForest.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("Random forest results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")


m5_regression=DecisionTreeRegressor(**{'criterion': 'mae', 'max_features': 'auto', 'presort': True, 'random_state': 30})
m5_regression.fit(X_train,y_train)
preds=m5_regression.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("M5 regression trees results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")

# to add optimal paraeters: 

GBDT=GradientBoostingRegressor(**{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'ls', 'max_depth': 6, 'max_features': 'auto', 'n_estimators': 160, 'random_state': 30})
GBDT.fit(X_train,y_train)
preds=GBDT.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("GBDT results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")

catBostReg=CatBoostRegressor(**{'depth': 3, 'l2_leaf_reg': 3, 'learning_rate': 0.3, 'verbose': 0})
catBostReg.fit(X_train,y_train)
preds=catBostReg.predict(X_test)
#scores:
mse=mean_squared_error(y_test,preds)
mae=mean_absolute_error(y_test,preds)
mape=mean_absolute_percentage_error(y_test,preds)
r2=r2_score(y_test,preds)
rmse=np.sqrt(mse)
print("CatBoostRegressor results on the testing data set:")
print("MSE:",mse)
print("RMSE: ",rmse)
print("MAE:",mae)
print("MAPE: ",mape)
print("R2: ",r2)
df=pd.DataFrame(
    {'lst1Title': y_test,
     'lst2Title': preds
    })
print("correlation between y_true and y_preds:",df["lst1Title"].corr(df["lst2Title"]))
print("---------------------------------------------------\n")



Cad concentration mean: 24.001398387096778
---------------------------------------------------

SVM with polinomial kernel, results on the testing data set:
MSE: 1530.9998520916963
RMSE:  39.12799320297038
MAE: 24.421083039918408
MAPE:  50.02881043784514
R2:  -0.34024218506314785
correlation between y_true and y_preds: 0.09342599216881639
---------------------------------------------------

SVM with RBF kernel, results on the testing data set:
MSE: 1457.4243440814873
RMSE:  38.17622747314731
MAE: 22.383201133833115
MAPE:  49.14535390753843
R2:  -0.27583394917206583
correlation between y_true and y_preds: 0.34369926398228245
---------------------------------------------------

SVM with linear kernel, results on the testing data set:
MSE: 1442.0554564161607
RMSE:  37.97440528061185
MAE: 23.115969264792593
MAPE:  59.85447930040693
R2:  -0.2623799755753826
correlation between y_true and y_preds: 0.286706379918632
---------------------------------------------------

Random forest results on